# Segmentation with U-net
## Best score 0.52563
Which has been obtained in this way:
- training the NN for 60 epochs with batchnormalization and dropout.
- we reach a plateau. The validation scorse was stable.
- so the last 20 epochs we use the dropout only at the last layer of the network, this increased the score reaching 0.52563

Other details in the specific cells.

In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
import os

# os.environ["CUDA_VISIBLE_DEVICES"]="-1" 
import tensorflow as tf
import numpy as np

# Set the seed for random operations. 
# This let our experiments to be reproducible. 
SEED = 1234
tf.random.set_seed(SEED)  

# Get current working directory
cwd = os.getcwd()

# Set GPU memory growth
# Allows to only as much GPU memory as needed
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

# Example: Image Segmentation

We decided to use only horizontal and vertical flip for data augumentation because changing too much the images led to worst predictions.

In [ ]:
# ImageDataGenerator
# ------------------

from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_img_data_gen = ImageDataGenerator(horizontal_flip=True,
                                        vertical_flip=True,
                                        fill_mode='reflect')

train_mask_data_gen = ImageDataGenerator(horizontal_flip=True,
                                         vertical_flip=True,
                                         fill_mode='reflect',
                                         rescale=1./255)

valid_img_data_gen = ImageDataGenerator()
valid_mask_data_gen = ImageDataGenerator(rescale=1./255)


In [ ]:
import shutil, random, json

RESET_VALIDATION = True                    # if True the split of train and validation is run again randomly, if False the split remain the same

DATASET_SPLIT = 0.8

path = '/kaggle/working/tmp'
if not os.path.exists(path) or RESET_VALIDATION:                # if the split doesn't exists OR if we want to create a new split, the new split is generated, otherwise not
    if os.path.exists(path):
        shutil.rmtree(path)
    if not os.path.exists(path):
        os.mkdir(path)
    if not os.path.exists(path+'/training'):
        os.mkdir(path+'/training')
    if not os.path.exists(path+'/validation'):
        os.mkdir(path+'/validation')

    source = "/kaggle/input/ann-and-dl-image-segmentation/Segmentation_Dataset/training"

    # Destination paths
    dest_train = path+'/training'
    dest_valid = path+'/validation'

    # dictionary to show the split with the format shown in the Evaluation tab
    dataset_split = {}  
    dataset_split["training"] = []
    dataset_split["validation"] = []

    if not os.path.exists(dest_train+'/'+'images'):
        os.mkdir(dest_train+'/'+'images')
    if not os.path.exists(dest_train+'/'+'images/img'):
        os.mkdir(dest_train+'/'+'images/img')
    if not os.path.exists(dest_valid+'/'+'images'):
        os.mkdir(dest_valid+'/'+'images')
    if not os.path.exists(dest_valid+'/'+'images/img'):
        os.mkdir(dest_valid+'/'+'images/img')


    if not os.path.exists(dest_train+'/'+'masks'):
        os.mkdir(dest_train+'/'+'masks')
    if not os.path.exists(dest_train+'/'+'masks/img'):
        os.mkdir(dest_train+'/'+'masks/img')
    if not os.path.exists(dest_valid+'/'+'masks'):
        os.mkdir(dest_valid+'/'+'masks')
    if not os.path.exists(dest_valid+'/'+'masks/img'):
        os.mkdir(dest_valid+'/'+'masks/img')

    #for images
    source_images = source+'/images/img'
    source_masks = source+'/masks/img'
    files = os.listdir(source_images)
    random.shuffle(files)
    #create training set randomly
    for i in range(int(len(files)*DATASET_SPLIT)):
        dest = shutil.copy(source_images+'/'+files[i], dest_train+'/images/img/'+files[i])
        dest = shutil.copy(source_masks+'/'+files[i], dest_train+'/masks/img/'+files[i])
        dataset_split["training"].append(files[i])
    #create validation set randomly
    for j in range(i+1, len(files)):
        dest = shutil.copy(source_images+'/'+files[j], dest_valid+'/images/img/'+files[j])
        dest = shutil.copy(source_masks+'/'+files[j], dest_valid+'/masks/img/'+files[j])
        dataset_split["validation"].append(files[j])

    # create the json file using the dictionary dataset_split
    with open('dataset_split.json', 'w') as fp:
        json.dump(dataset_split, fp)

print("Images uploaded for training:", len(os.listdir(dest_train+'/images/img')))
print("Images uploaded for validation:", len(os.listdir(dest_valid+'/images/img')))

In [ ]:
# Create generators to read images from dataset directory
# -------------------------------------------------------
dataset_dir = '/kaggle/working/tmp'
########################################changed batch size 12 from 4
# Batch size
bs = 32

# img shape
img_h = 256
img_w = 256

num_classes=2

classes = ['background',    # 0
           'building',      # 1
          ]


training_dir = os.path.join(dataset_dir, 'training')

train_img_gen = train_img_data_gen.flow_from_directory(os.path.join(training_dir, 'images'),
                                                       target_size=(img_h, img_w),
                                                       batch_size=bs, 
                                                       class_mode=None, # Because we have no class subfolders in this case
                                                       shuffle=True,
                                                       interpolation='bilinear',
                                                       seed=SEED)  

train_mask_gen = train_mask_data_gen.flow_from_directory(os.path.join(training_dir, 'masks'),
                                                         target_size=(img_h, img_w),
                                                         batch_size=bs,
                                                         class_mode=None, # Because we have no class subfolders in this case
                                                         shuffle=True,
                                                         interpolation='bilinear',
                                                         color_mode = 'grayscale',
                                                         seed=SEED)
train_gen = zip(train_img_gen, train_mask_gen)

validation_dir = os.path.join(dataset_dir, 'validation')
valid_img_gen = valid_img_data_gen.flow_from_directory(os.path.join(validation_dir, 'images'),
                                                       target_size=(img_h, img_w),
                                                       batch_size=bs, 
                                                       class_mode=None, # Because we have no class subfolders in this case
                                                       shuffle=False,
                                                       interpolation='bilinear',
                                                       seed=SEED)
valid_mask_gen = valid_mask_data_gen.flow_from_directory(os.path.join(validation_dir, 'masks'),
                                                         target_size=(img_h, img_w),
                                                         batch_size=bs, 
                                                         class_mode=None, # Because we have no class subfolders in this case
                                                         shuffle=False,
                                                         interpolation='bilinear',
                                                         color_mode = 'grayscale',
                                                         seed=SEED)
valid_gen = zip(valid_img_gen, valid_mask_gen)


In [ ]:
# Create Dataset objects
# ----------------------

# Training
# --------
train_dataset = tf.data.Dataset.from_generator(lambda: train_gen,
                                               output_types=(tf.float32, tf.float32),
                                               output_shapes=([None, img_h, img_w, 3], [None, img_h, img_w, 1]))

def prepare_target(x_, y_):
    y_ = tf.cast(y_, tf.float32)
    return x_, y_

train_dataset = train_dataset.map(prepare_target)

# Repeat
train_dataset = train_dataset.repeat()

# Validation
# ----------
valid_dataset = tf.data.Dataset.from_generator(lambda: valid_gen, 
                                               output_types=(tf.float32, tf.float32),
                                               output_shapes=([None, img_h, img_w, 3], [None, img_h, img_w, 1]))
valid_dataset = valid_dataset.map(prepare_target)

# Repeat
valid_dataset = valid_dataset.repeat()

# Test
# ----
test_dataset = tf.data.Dataset.from_generator(lambda: test_gen,
                                              output_types=(tf.float32, tf.float32),
                                              output_shapes=([None, img_h, img_w, 3], [None, img_h, img_w, 1]))
test_dataset = test_dataset.map(prepare_target)

# Repeat
test_dataset = valid_dataset.repeat()

## Convolutional Neural Network (CNN)
### Encoder-Decoder
Here we decide to implement a U-Net which is recommended for segmentation problems. At first using only dropout, than we realized that a batch normalization led to better score.

In [ ]:
from keras.models import Model, load_model
from keras.layers import Input, BatchNormalization, Activation, Dense, Dropout
from keras.layers.core import Lambda, RepeatVector, Reshape
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers.pooling import MaxPooling2D, GlobalMaxPool2D
from keras.layers.merge import concatenate, add
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

# Build U-Net model
inputs = tf.keras.layers.Input((img_h, img_w, 3))
s = tf.keras.layers.Lambda(lambda x: x / 255)(inputs)

c1 = tf.keras.layers.Conv2D(16, (3, 3), kernel_initializer='he_normal',
                            padding='same')(s)
c1 = tf.keras.layers.BatchNormalization()(c1)
c1 = tf.keras.layers.Activation("relu")(c1)
#c1 = tf.keras.layers.Dropout(0.1)(c1)

c1 = tf.keras.layers.Conv2D(16, (3, 3), kernel_initializer='he_normal',
                            padding='same')(c1)

p1 = tf.keras.layers.MaxPooling2D((2, 2))(c1)

c2 = tf.keras.layers.Conv2D(32, (3, 3), kernel_initializer='he_normal',
                            padding='same')(p1)
c2 = tf.keras.layers.BatchNormalization()(c2)
c2 = tf.keras.layers.Activation("relu")(c2)
#c2 = tf.keras.layers.Dropout(0.1)(c2)


c2 = tf.keras.layers.Conv2D(32, (3, 3), kernel_initializer='he_normal',
                            padding='same')(c2)

p2 = tf.keras.layers.MaxPooling2D((2, 2))(c2)

c3 = tf.keras.layers.Conv2D(64, (3, 3), kernel_initializer='he_normal',
                            padding='same')(p2)
c3 = tf.keras.layers.BatchNormalization()(c3)
c3 = tf.keras.layers.Activation("relu")(c3)
#c3 = tf.keras.layers.Dropout(0.2)(c3)

c3 = tf.keras.layers.Conv2D(64, (3, 3), kernel_initializer='he_normal',
                            padding='same')(c3)

p3 = tf.keras.layers.MaxPooling2D((2, 2))(c3)

c4 = tf.keras.layers.Conv2D(128, (3, 3), kernel_initializer='he_normal',
                            padding='same')(p3)
c4 = tf.keras.layers.BatchNormalization()(c4)
c4 = tf.keras.layers.Activation("relu")(c4)
#c4 = tf.keras.layers.Dropout(0.2)(c4)

c4 = tf.keras.layers.Conv2D(128, (3, 3), kernel_initializer='he_normal',
                            padding='same')(c4)

p4 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(c4)

c5 = tf.keras.layers.Conv2D(256, (3, 3), kernel_initializer='he_normal',
                            padding='same')(p4)    
c5 = tf.keras.layers.BatchNormalization()(c5)
c5 = tf.keras.layers.Activation("relu")(c5)
#c5 = tf.keras.layers.Dropout(0.3)(c5)

c5 = tf.keras.layers.Conv2D(256, (3, 3), kernel_initializer='he_normal',
                            padding='same')(c5)

u6 = tf.keras.layers.Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c5)
u6 = tf.keras.layers.concatenate([u6, c4])
c6 = tf.keras.layers.Conv2D(128, (3, 3), kernel_initializer='he_normal',
                            padding='same')(u6)
c6 = tf.keras.layers.BatchNormalization()(c6)
c6 = tf.keras.layers.Activation("relu")(c6)
#c6 = tf.keras.layers.Dropout(0.2)(c6)
c6 = tf.keras.layers.Conv2D(128, (3, 3), kernel_initializer='he_normal',
                            padding='same')(c6)

u7 = tf.keras.layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c6)
u7 = tf.keras.layers.concatenate([u7, c3])
c7 = tf.keras.layers.Conv2D(64, (3, 3), kernel_initializer='he_normal',
                            padding='same')(u7)
c7 = tf.keras.layers.BatchNormalization()(c7)
c7 = tf.keras.layers.Activation("relu")(c7)
#c7 = tf.keras.layers.Dropout(0.2)(c7)
c7 = tf.keras.layers.Conv2D(64, (3, 3), kernel_initializer='he_normal',
                            padding='same')(c7)

u8 = tf.keras.layers.Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(c7)
u8 = tf.keras.layers.concatenate([u8, c2])
c8 = tf.keras.layers.Conv2D(32, (3, 3), kernel_initializer='he_normal',
                            padding='same')(u8)
#c8 = tf.keras.layers.Dropout(0.1)(c8)
c8 = tf.keras.layers.Conv2D(32, (3, 3), kernel_initializer='he_normal',
                            padding='same')(c8)

u9 = tf.keras.layers.Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same')(c8)
u9 = tf.keras.layers.concatenate([u9, c1], axis=3)
c9 = tf.keras.layers.Conv2D(16, (3, 3), kernel_initializer='he_normal',
                            padding='same')(u9)  
c9 = tf.keras.layers.BatchNormalization()(c9)
c9 = tf.keras.layers.Activation("relu")(c9)
c9 = tf.keras.layers.Dropout(0.2)(c9)

c9 = tf.keras.layers.Conv2D(16, (3, 3), kernel_initializer='he_normal',
                            padding='same')(c9)       
c9 = tf.keras.layers.BatchNormalization()(c9)
c9 = tf.keras.layers.Activation("relu")(c9)

outputs = tf.keras.layers.Conv2D(1, (1, 1), activation='sigmoid')(c9)

model = tf.keras.Model(inputs=[inputs], outputs=[outputs])
model.summary()

## Prepare the model for training
We saw increasing score using:
- dice_loss for the loss
- Adam optimizer with learning rate 0.0001
- IoU metric

In [ ]:
def dice_loss(y_true, y_pred):
    numerator = 2 * tf.reduce_sum(y_true * y_pred, axis=(1,2,3))
    denominator = tf.reduce_sum(y_true + y_pred, axis=(1,2,3))
    return 1 - numerator / denominator

# learning rate
lr = 1e-4
optimizer = tf.keras.optimizers.Adam(learning_rate=lr)

# -------------------

# Validation metrics
# ------------------

def my_IoU(y_true, y_pred):
    # from pobability to predicted class {0, 1}
    y_pred = tf.cast(y_pred > 0.5, tf.float32)              # when using sigmoid. Use argmax for softmax

    # A and B
    intersection = tf.reduce_sum(y_true * y_pred)
    # A or B
    union = tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) - intersection
    # IoU
    return intersection / union

metrics = [my_IoU]
# ------------------

# Compile Model

model.compile(optimizer=optimizer, loss=dice_loss, metrics=metrics)
model.summary()

## Training with callbacks
To have better scores we saved the weights, and re-train the net using the old weights. Our best score is 0.52563 which has been obtained in this way:
- training the net 60 times with batchnormalization and dropout.
- we reach a plateau. The validation scorse was stable.
- so the last 20 epochs we use the dropout only at the last layer of the network, this increased the score reaching 0.52563

In [ ]:
import os
from datetime import datetime

cwd = os.getcwd()

exps_dir = os.path.join(cwd, 'segmentation_experiments')
if not os.path.exists(exps_dir):
    os.makedirs(exps_dir)

now = datetime.now().strftime('%b%d_%H-%M-%S')

model_name = 'CNN'

exp_dir = os.path.join(exps_dir, model_name + '_' + str(now))
if not os.path.exists(exp_dir):
    os.makedirs(exp_dir)

callbacks = []

# Model checkpoint
# ----------------
ckpt_dir = os.path.join(exp_dir, 'ckpts')
if not os.path.exists(ckpt_dir):
    os.makedirs(ckpt_dir)

ckpt_callback = tf.keras.callbacks.ModelCheckpoint(filepath=os.path.join(ckpt_dir, 'cp_{epoch:02d}.ckpt'), 
                                                   save_weights_only=True)  # False to save the model directly
callbacks.append(ckpt_callback)

# Visualize Learning on Tensorboard
# ---------------------------------
tb_dir = os.path.join(exp_dir, 'tb_logs')
if not os.path.exists(tb_dir):
    os.makedirs(tb_dir)

# By default shows losses and metrics for both training and validation
tb_callback = tf.keras.callbacks.TensorBoard(log_dir=tb_dir,
                                             profile_batch=0,
                                             histogram_freq=0)  # if 1 shows weights histograms
callbacks.append(tb_callback)

# Early Stopping
# --------------
early_stop = False
if early_stop:
    es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)
    callbacks.append(es_callback)

if 'my_model_weights.h5' in os.listdir('/kaggle/working'):
    model.load_weights('my_model_weights.h5')

history = model.fit(x=train_dataset,
          epochs=60,  #### set repeat in training dataset
          steps_per_epoch=len(train_img_gen),
          validation_data=valid_dataset,
          validation_steps=len(valid_img_gen), 
          callbacks=callbacks)

saved_weights = model.save_weights('my_model_weights.h5')

# How to visualize Tensorboard

# 1. tensorboard --logdir EXPERIMENTS_DIR --port PORT     <- from terminal
# 2. localhost:PORT   <- in your browser

## Test model

## Compute prediction

In [ ]:
import time
import matplotlib.pyplot as plt
from IPython.display import clear_output
from PIL import Image

%matplotlib notebook

# define funct that riceive the path of an image and compute the groundtruth   
def predict_ground_truth(image_path):
    img = Image.open(image_path)
    img = img.resize((img_h, img_w))

    img_arr = np.expand_dims(np.array(img), 0)

    out_sigmoid = model.predict(x=img_arr / 1.)

    # Get predicted class as the index corresponding to the maximum value in the vector probability
    predicted_class = np.ones(shape=(img_w, img_h), dtype=np.int64)

    for j in range(len(out_sigmoid[0])):
        for k in range(len(out_sigmoid[0][j])):
            if out_sigmoid[0][j][k][0] > 0.5:
                predicted_class[j][k] = 1
            else:
                predicted_class[j][k] = 0


    prediction_img = np.zeros([img_h, img_w])

    prediction_img[np.where(predicted_class == 0)] = colors_dict[0]
    prediction_img[np.where(predicted_class == 1)] = colors_dict[1]

    return img_arr[0, ...], prediction_img

In [ ]:
import os
from datetime import datetime

#function to create csv file
def create_csv(results, results_dir='./'):
    csv_fname = 'results_'
    csv_fname += datetime.now().strftime('%b%d_%H-%M-%S') + '.csv'

    with open(csv_fname, 'w') as f:
        f.write('ImageId,EncodedPixels,Width,Height\n')

        for key, value in results.items():
            f.write(key + ',' + str(value) + ',' + '256' + ',' + '256' + '\n')

#function to encode predicted ground truth in the required format
def rle_encode(img):
    # Flatten column-wise
    pixels = img.T.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

In [ ]:
# Cycle over test images    
test_dir = "/kaggle/input/ann-and-dl-image-segmentation/Segmentation_Dataset/test"
test_img_dir = os.path.join(test_dir, 'images', 'img')

img_filenames = os.listdir(test_img_dir)

#fig, ax = plt.subplots(1, 2, figsize=(8, 8))
#fig.show()

# Assign a color to each class
colors_dict = {}
colors_dict[0] = 0          # background
colors_dict[1] = 255       # foreground

results = {}
counter = 1
perc = 0

for img_filename in img_filenames:
#if 1:
  #  img_filename = '492.tif'

    #print % of test done
    if(counter>perc*len(img_filenames)/100):
        clear_output()
        print('-'*perc, '', perc,'%')
        perc = perc + 1
    counter = counter + 1

    #prediction of ground truth of the image
    img, pred = predict_ground_truth(os.path.join(test_img_dir, img_filename))
    pred = np.uint8(pred/255)

    results[img_filename[:-4]] = rle_encode(pred)

    #ax[0].imshow(np.uint8(img), cmap='gray')
    #ax[1].imshow(pred, cmap='gray')

    #fig.canvas.draw()
    #time.sleep(1)

create_csv(results)

In [ ]:
from IPython.display import FileLink, FileLinks
FileLinks(".")